<h1 style="color: blue">Question 1</h1>

### 0. Import fundamental modules

In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.image as mpimg

### 1. Retrieve and load mnist

In [2]:
df_mnist = pd.read_csv('C:/Users/Balu/Desktop//mnist_784.csv')
X = df_mnist.drop('class', axis=1)
y = df_mnist['class']

FileNotFoundError: [Errno 2] No such file or directory: './mnist_784.csv'

### 2. Display digits from 0 to 9

In [ ]:
fig, axes = plt.subplots(1, 10)
fig.set_figheight(5)
fig.set_figwidth(20)

for digit in range(10):
    # get index of digit
    idx = list(y).index(digit)
    
    # reshape values and plot
    digit_image = X.iloc[idx].values.reshape(28,28)
    axes[digit].imshow(digit_image, cmap="binary")
    axes[digit].axis('off')

### 3. Use PCA to retrieve 1st and 2nd PC and output explained variance ratio

In [ ]:
from sklearn.decomposition import PCA

# create PCA instance with two PCs
pca = PCA(n_components = 2)
X2D = pca.fit_transform(X)    
df_X2D = pd.DataFrame(X2D)

# retrieve 1th and 2nd PCs
first_PC = df_X2D[0]
second_PC = df_X2D[1]
print('First PC: \n', first_PC)
print('Second PC: \n', second_PC)

# get explained variance ratio
explained_variance_ratio = pca.explained_variance_ratio_
print('Explained Variance Ratio: ', explained_variance_ratio)

### 4. Plot the projections of PCs onto a 1D hyperplance

In [ ]:
# set range of horizontal line
horizontal_min = min(first_PC) - 500
horizontal_max = max(first_PC) + 500

# project first PC onto 1D hyperplane
plt.figure()
plt.hlines(0, horizontal_min, horizontal_max, colors='black')
plt.scatter(first_PC, np.zeros((1, 70000)), c='blue', alpha=0.1)
plt.show()

# project second PC onto 1D hyperplane    
plt.figure()
plt.hlines(0, horizontal_min, horizontal_max, colors='black')
plt.scatter(second_PC, np.zeros((1, 70000)), c='green', alpha=0.1)
plt.show()

### 5. Use incremental PCA to reduce the dimensionality to 154

In [ ]:
from sklearn.decomposition import IncrementalPCA

n_batches = 100
inc_pca = IncrementalPCA(n_components=154)
for X_batch in np.array_split(X, n_batches):
    inc_pca.partial_fit(X_batch)

X_reduced = inc_pca.transform(X)

### 6. Display the original and compressed digits

In [ ]:
#decompress reduced data to 784 dimentions
X_recovered = pd.DataFrame(inc_pca.inverse_transform(X_reduced))

fig, axes = plt.subplots(2, 10)
fig.set_figheight(5)
fig.set_figwidth(20)

for digit in range(10):
    # get index of digit
    idx = list(y).index(digit)
    
    # reshape values and plot original digit
    idx = list(y).index(digit)
    digit_image = X.iloc[idx].values.reshape(28,28)
    axes[0, digit].imshow(digit_image, cmap="binary")
    axes[0, digit].axis('off')
    
    # reshape values and plot original digit
    idx = list(y).index(digit)
    digit_image = X_recovered.iloc[idx].values.reshape(28,28)
    axes[1, digit].imshow(digit_image, cmap="binary")
    axes[1, digit].axis('off')

<h1 style="color:blue">Question 2</h1>

### 1. Generate Swiss roll dataset.

In [ ]:
from sklearn.datasets import make_swiss_roll
X, y = make_swiss_roll(1500)

### 2. Plot the resulting generated Swiss

In [ ]:
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(projection='3d')
ax.scatter3D(X[:,0], X[:,1], X[:,2], c=y, cmap='Spectral')
ax.set_title('Swiss roll')

### 3. Use kPCA with linear, RBF, and sigmoid kernels

In [ ]:
from sklearn.decomposition import KernelPCA

# linear kernel
linear_pca = KernelPCA(n_components = 2, kernel="linear")
X_linear = pd.DataFrame(linear_pca.fit_transform(X))

# rbf kernel
rbf_pca = KernelPCA(n_components = 2, kernel='rbf', gamma=0.04)
X_rbf = pd.DataFrame(rbf_pca.fit_transform(X))

# sigmoid kernel
sigmoid_pca = KernelPCA(n_components = 2, kernel='sigmoid', gamma=0.001, coef0=1)
X_sigmoid = pd.DataFrame(sigmoid_pca.fit_transform(X))

### 4. Plot the kPCA results

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15,5))

# linear kernel
axes[0].scatter(X_linear[0], X_linear[1], c=y, cmap='Spectral')
axes[0].title.set_text('Linear Kernel')

# rbf kernel
axes[1].scatter(X_rbf[0], X_rbf[1], c=y, cmap='Spectral')
axes[1].title.set_text('Rbf Kernel')

# sigmoid kernel
axes[2].scatter(X_sigmoid[0], X_sigmoid[1], c=y, cmap='Spectral')
axes[2].title.set_text('sigmoid Kernel')

### 5. Apply Logistic Regression for classification

In [ ]:
# Compute unstructured hierarchical clustering
from sklearn.cluster import AgglomerativeClustering

ward = AgglomerativeClustering(n_clusters=5, linkage='ward').fit(X)
label = ward.labels_

# Apply kPCA and Logistic Regression for Grid search
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline


full_pipeline = Pipeline([
        ("kpca", KernelPCA(n_components=2)),
        ("lr", LogisticRegression(solver='lbfgs', max_iter=500))
    ])

param_grid = [{
        "kpca__gamma": np.linspace(0.3, 0.05, 10),
        "kpca__kernel": ["linear", "rbf", "sigmoid"],
    }]

grid_search = GridSearchCV(full_pipeline, param_grid)
grid_search.fit(X, label)

# Best parametors and estimator
best_parameters = grid_search.best_params_
best_estimator = grid_search.best_estimator_

print('Best parameters: ', best_parameters)
print('Best estimator: ', best_estimator)

### 6. Plot the results using GridSearchCV

In [ ]:
y_pred = best_estimator.predict(X)

fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(projection='3d')
ax.scatter3D(X[:,0], X[:,1], X[:,2], c=y_pred, cmap='Spectral')
ax.set_title('Swiss roll clustering with best estimator')
plt.show()

plt.scatter(X[:,0], X[:,2], c=y_pred, cmap='Spectral')
plt.title('Swiss roll clustering in 2D with best estimator ')